# News Articles Scraping

The aim of this Notebook is to scrape news articles from https://assets.msn.com/labs/mind website using Python.

In [1]:
#import necessary libraries
import pandas as pd

Load MIND news dataset

In [2]:
#Load MIND news dataset
DATA_PATH='./dataset/'

df = pd.read_csv(os.path.join(DATA_PATH,'news.tsv'),sep = '\t',header=None) 

In [3]:
#Add names to dataFrame colums
df.columns=['News ID',
"Category",
"SubCategory",
"Title",
"Abstract",
"URL",
"Title Entities",
"Abstract Entities "]

In [4]:
#Show the head of data
df.head()

,News ID,Category,SubCategory,Title,Abstract,URL,Title Entities,Abstract Entities
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."
...,...,...,...,...,...,...,...,...
51277,N16909,weather,weathertopstories,"Adapting, Learning And Soul Searching: Reflect...",Woolsey Fire Anniversary: A community is forev...,https://assets.msn.com/labs/mind/BBWzQJK.html,"[{""Label"": ""Woolsey Fire"", ""Type"": ""N"", ""Wikid...","[{""Label"": ""Woolsey Fire"", ""Type"": ""N"", ""Wikid..."
51278,N47585,lifestyle,lifestylefamily,Family says 13-year-old Broadway star died fro...,NaN,https://assets.msn.com/labs/mind/BBWzQYV.html,"[{""Label"": ""Broadway theatre"", ""Type"": ""F"", ""W...",[]
51279,N7482,sports,more_sports,St. Dominic soccer player tries to kick cancer...,"Sometimes, what happens on the sidelines can b...",https://assets.msn.com/labs/mind/BBWzQnK.html,[],[]
51280,N34418,sports,soccer_epl,How the Sounders won MLS Cup,"Mark, Jeremiah and Casey were so excited they ...",https://assets.msn.com/labs/mind/BBWzQuK.html,"[{""Label"": ""MLS Cup"", ""Type"": ""U"", ""WikidataId...",[]


Missing values analysis

In [5]:
# Checking for null values
df.isnull().sum()

News ID                  0
Category                 0
SubCategory              0
Title                    0
Abstract              2666
URL                      0
Title Entities           3
Abstract Entities        4
dtype: int64

In [6]:
#Droping Null Values
df.dropna(inplace=True)

Now let’s scrape!

First, we’ll turn the page content into a BeautifulSoup object, which will allow us to parse the HTML tags.

In [16]:
#importing the necessary packages
import requests
from bs4 import BeautifulSoup

Then, we’ll need to figure out which HTML tags contain the headline and the main text of the article.

To do this, we’ll use Google Chrome’s Inspect tool. 
- Open any news article link we chose from our dataset's urls  in a new tab, right-click on the page and choose Inspect from the drop-down menu.


- Click on the little button outlined above to find the HTML tags corresponding to anything you see on the page.


- When you hover over the text on the page you want to select, in this case the headline and the main     text of the article, you’ll see which HTML tags are used to identify that text.

The main text of the article is surrounded by the \<p> tag. This time we’ll have to find all of the \<p> tags contained on the page since the paragraphs of the article are each contained in a \<p> tag.

In [17]:
urls = df['URL'].to_list()


In [18]:
def scrape_from_url(url):
    response = requests.get(url)
# Turn page into BeautifulSoup object to access HTML tags 
    soup = BeautifulSoup(response.text, 'html.parser')
# Get text from all <p> tags.
    articles = soup.find_all('p')
# Get the text from each of the “p” tags and strip surrounding whitespace.
    content = ""
    for article in articles:
        content += article.get_text()

    result = content.encode('ascii', errors='ignore').decode()
    return result

In [19]:
article_dict = {}
for url in urls:
    article_dict[url] = scrape_from_url(url)


In [20]:
#Creating dataFrame for URL and its corresponding article
article_df=pd.DataFrame(list(article_dict.items()), columns=['URL', 'Article'])

In [21]:
#Merge the origiinal dataFrame with  url-article 
df=pd.merge(df,article_df,on="URL")



In [28]:
#Saving the scrapped version to pickle file
df=df[['News ID', 'Category', 'SubCategory', 'Title', 'Abstract', 'URL' , 'Article']]
df.to_pickle("goodDataWithTrafilatura.p")

In [1]:
from newspaper import Article


In [ ]:
Article()